In [14]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from datetime import date

In [65]:
# Grupo 1
def listar_publicacoes(data):
    
    options = Options()
    options.add_argument('--headless')
    
    driver = webdriver.Chrome(options=options)

    url = 'https://www.in.gov.br/leiturajornal'

    data = data
    orgao = 'Ministério-da-Saúde'
    orgao_sub = 'Agência-Nacional-de-Vigilância-Sanitária'
    ato = 'Resolução'

    url_base = f'https://www.in.gov.br/leiturajornal?secao=dou1&data={data}&org={orgao}&org_sub={orgao_sub}&ato={ato}'

    driver.get(url_base)
               
    html_content = driver.page_source

    driver.quit()

    soup = BeautifulSoup(html_content, 'html.parser')

    padrao = re.compile(r'/web/dou/-/resolucao.*')
    links = soup.find_all('a', href=padrao)

    links_filtrados = [] 
    for link in links:
        li = link.get('href')
        links_filtrados.append(f'https://www.in.gov.br/{li}')
       
        
    return(links_filtrados)

In [66]:
def extrair_informacoes(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Grupo 2
    publicado_em_elemento = soup.find('span', {'class': 'publicado-dou-data'})
    publicado_em = publicado_em_elemento.text.strip() if publicado_em_elemento is not None else None
    
    edicao_elemento = soup.find('span', {'class': 'edicao-dou-data'})
    edicao = edicao_elemento.text.strip() if edicao_elemento is not None else None
    
    secao_elemento = soup.find('span', {'class': 'secao-dou'})
    secao = secao_elemento.text.strip() if secao_elemento is not None else None
    
    pagina_elemento = soup.find('span', {'class': 'secao-dou-data'})
    pagina = pagina_elemento.text.strip() if pagina_elemento is not None else None
    
    orgao_elemento = soup.find('span', {'class': 'orgao-dou-data'})
    orgao = orgao_elemento.text.strip() if orgao_elemento is not None else None
    
    assina_elemento = soup.find('p', {'class': 'assina'})
    assina = assina_elemento.text.strip() if assina_elemento is not None else None
    
    identifica_elemento = soup.find('p', {'class': 'identifica'})
    identifica = identifica_elemento.text.strip() if identifica_elemento is not None else None
    
    preambulo_elemento = soup.find('p', {'class': 'dou-paragraph'})
    preambulo = preambulo_elemento.text.strip() if preambulo_elemento is not None else None
    
    versao_oficial_elemento = soup.find('a', {'id': 'versao-certificada'})
    versao = versao_oficial_elemento.get('href') if versao_oficial_elemento is not None else None
    
    artigos = re.compile(r'Art\.\s?\d+º')
    paragrafos_artigos = []

    for paragraph in soup.find_all('p'):
        if artigos.search(paragraph.text):
            paragrafos_artigos.append(paragraph.text.strip())

    #Grupo 3
    #Extrair número do Expediente
    expediente_elements = soup.find_all('p', text=re.compile(r'Expediente'))
    numeros_expediente = []
    for expediente_element in expediente_elements:
        expediente_text = expediente_element.text.strip()
        numero_expediente = re.search(r'(\d+/\d+-\d)', expediente_text)
        if numero_expediente:
            numeros_expediente.append(numero_expediente.group(1))

    #Extrair Texto completo
    completo = []
    for i in soup.find_all('p', {'class':'dou-paragraph'}):
        completo.append(i.text.strip())
       
    #Ajuste fino
       
    data = {
    'DT_PUBLICACAO': [publicado_em],
    'EDICAO': [edicao],
    'SECAO': [secao],
    'PAGINA': [pagina],
    'ORGAO':[orgao],
    'ASSINATURA':[assina],
    'PREAMBULO':[preambulo],
    'ARTIGOS':[paragrafos_artigos],
    'IDENTIFICACAO':[identifica], 
    'VERSAO_OFICIAL': [versao],
    'EXPEDIENTE': [numeros_expediente],
    'TEXTO_COMPLETO':[completo]
    }

    df = pd.DataFrame(data)
    return(df)

In [67]:
publicacoes = listar_publicacoes(date.today().strftime("%d-%m-%Y"))

In [62]:
dados = pd.DataFrame()

for i in publicacoes:
    link = extrair_informacoes(i)
    link = link.reset_index(drop=True)  
    dados = pd.concat([dados, link], axis=0)
    
dados = dados.reset_index(drop=True) #Redefinir índice
dados.to_excel(f'{date.today()}.xlsx',index=False)

In [63]:
dados

,DT_PUBLICACAO,EDICAO,SECAO,PAGINA,ORGAO,ASSINATURA,PREAMBULO,ARTIGOS,IDENTIFICACAO,VERSAO_OFICIAL,EXPEDIENTE,TEXTO_COMPLETO
0,26/06/2023,119,Seção: 1,114,Ministério da Saúde/Agência Nacional de Vigilância Sanitária/2ª Diretoria/Gerência-Geral de Alimentos,PATRÍCIA FERNANDES NANTES DE CASTILHO,"A Gerente-Geral de Alimentos, no uso das atribuições que lhe confere o art. 96, aliado ao art. 203, I, §1º do Regimento Interno aprovado pela Resolução de Diretoria Colegiada-RDC nº 585, de 10 de dezembro de 2021, resolve:","[Art.1º Deferir as petições relacionadas à Gerência-Geral de Alimentos, conforme relação anexa., Art. 2º Esta Resolução entra em vigor na data de sua publicação.]","RESOLUÇÃO-RE Nº 2.276, DE 23 DE JUNHO DE 2023",http://pesquisa.in.gov.br/imprensa/jsp/visualiza/index.jsp?data=26/06/2023&jornal=515&pagina=114,[],"[A Gerente-Geral de Alimentos, no uso das atribuições que lhe confere o art. 96, aliado ao art. 203, I, §1º do Regimento Interno aprovado pela Resolução de Diretoria Colegiada-RDC nº 585, de 10 de dezembro de 2021, resolve:, Art.1º Deferir as petições relacionadas à Gerência-Geral de Alimentos, conforme relação anexa., Art. 2º Esta Resolução entra em vigor na data de sua publicação., Relatório de Conferência - Alimentos: 429923, NOME DA EMPRESA / CNPJ, NOME DO PRODUTO, NÚMERO DO PROCESSO / REGISTRO, PETIÇÃO(ÕES) / EXPEDIENTE(S), --------------------------------------, AIRELA INDÚSTRIA FARMACÊUTICA LTDA. / 01.858.973/0001-29, SUPLEMENTO ALIMENTAR EM CÁPSULAS COM LACTOBACILLUS ACIDOPHILUS NCFM, 25351.237856/2021-12 / 675310001, 457 - Inclusão de Marca / 0155892/23-1, --------------------..."
1,26/06/2023,119,Seção: 1,115,Ministério da Saúde/Agência Nacional de Vigilância Sanitária/2ª Diretoria/Gerência-Geral de Alimentos,PATRÍCIA FERNANDES NANTES DE CASTILHO,"A Gerente-Geral de Alimentos, no uso das atribuições que lhe confere o art. 96, aliado ao art. 203, I, §1º do Regimento Interno aprovado pela Resolução de Diretoria Colegiada-RDC nº 585, de 10 de dezembro de 2021, resolve:","[Art.1º Deferir as petições de avaliação relacionadas à Gerência-Geral de Alimentos, conforme relação anexa., Art. 2º Esta Resolução entra em vigor na data de sua publicação.]","RESOLUÇÃO-RE Nº 2.277, DE 23 DE JUNHO DE 2023",http://pesquisa.in.gov.br/imprensa/jsp/visualiza/index.jsp?data=26/06/2023&jornal=515&pagina=115,[],"[A Gerente-Geral de Alimentos, no uso das atribuições que lhe confere o art. 96, aliado ao art. 203, I, §1º do Regimento Interno aprovado pela Resolução de Diretoria Colegiada-RDC nº 585, de 10 de dezembro de 2021, resolve:, Art.1º Deferir as petições de avaliação relacionadas à Gerência-Geral de Alimentos, conforme relação anexa., Art. 2º Esta Resolução entra em vigor na data de sua publicação., RELATÓRIO DE CONFERÊNCIA - ALIMENTOS: 419123, NOME DA EMPRESA / CNPJ, NOME DO PRODUTO, NÚMERO DO PROCESSO, PETIÇÃO(ÕES) / EXPEDIENTE(S), --------------------------------------------------------, ABIFITO ASSOCIACAO BRASILEIRA DA INDUSTRIA FITOTERAPICA /03.986.187/0001-32, ROOIBOS, 25351.052850/2023-39, 4134 - AVALIAÇÃO DE SEGURANÇA DE NOVOS ALIMENTOS IN NATURA, MINIMAMENTE PROCESSADOS OU DESIDR..."
2,26/06/2023,119,Seção: 1,115,Ministério da Saúde/Agência Nacional de Vigilância Sanitária/2ª Diretoria/Gerência-Geral de Alimentos,PATRÍCIA FERNANDES NANTES DE CASTILHO,"A Gerente-Geral de Alimentos, no uso das atribuições que lhe confere o art. 96, aliado ao art. 203, I, §1º do Regimento Interno aprovado pela Resolução de Diretoria Colegiada-RDC nº 585, de 10 de dezembro de 2021, resolve:","[Art.1º Indeferir as petições de avaliação relacionadas à Gerência-Geral de Alimentos, conforme relação anexa., Art. 2º Esta Resolução entra em vigor na data de sua publicação.]","RESOLUÇÃO-RE Nº 2.278, DE 23 DE JUNHO DE 2023",http://pesquisa.in.gov.br/imprensa/jsp/visualiza/index.jsp?data=26/06/2023&jornal=515&pagina=115,[],"[A Gerente-Geral de Alimentos, no uso das atribuições que lhe confere o art. 96, aliado ao art. 203, I, §1º do Regimen